In [1]:
import sys
sys.path.append("..")

%load_ext autoreload
%autoreload 2

In [68]:
import warnings
warnings.filterwarnings("ignore")

In [41]:
from irt_lib.editorial_data_helper import create_full_data

from irt_lib.helpers import split_by_doc_id, do_metrics, run_full_cv

In [3]:
style_data = create_full_data(feature_types=['style_scaled'])

In [4]:
style_data.head()

,feats,user_id,label,doc_id
0,"[0.9675265502110052, -1.2649550896245798, 2.21...",3,True,1638699
1,"[0.5305433924314203, -1.0457320355012902, 1.96...",3,True,1640113
2,"[1.5755031175565148, 0.4355048166830996, -0.63...",3,False,1640630
3,"[0.036562431463193934, -1.393328950147228, -0....",3,True,1640915
4,"[0.9675265502110052, -1.2649550896245798, 2.21...",2,True,1638699


In [12]:
from irt_lib.models import IRTNet

In [62]:
D_in = len(style_data.iloc[0].feats)
num_users = style_data.user_id.nunique()

In [70]:
model_arguments = {'D_in': D_in, 'num_users': num_users}
train_arguments = {'num_train_epochs': 200, 'learning_rate': 0.001}

In [71]:
model_cls = IRTNet

In [80]:
final_results = {}

In [ ]:
for C in [1e-3, 1e-4, 1e-5, 1e-6]:
    for reg_type in ['l1', 'l2']:
        for learning_rate in [0.01, 0.005]:
            model_arguments['C'] = C
            model_arguments['reg_type'] = reg_type
            train_arguments['learning_rate'] = learning_rate
            train_arguments['num_train_epochs'] = 100
            results = run_full_cv(style_data, model_cls, model_arguments, train_arguments, averaged=True)
            
            final_results[(C, reg_type, learning_rate)] = results

{'loss': 0.526465283203125, 'learning_rate': 0.0031972789115646263, 'epoch': 68.02721088435374}
{'epoch': 100.0}
Finished round {'eval_loss': 0.5471543669700623, 'eval_macro_f1': 0.6759811616954474, 'eval_micro_f1': 0.7440476190476191, 'eval_pos_f1': 0.8244897959183672, 'eval_accuracy': 0.7440476190476191}
{'loss': 0.527666357421875, 'learning_rate': 0.0031972789115646263, 'epoch': 68.02721088435374}
{'epoch': 100.0}
Finished round {'eval_loss': 0.5416099429130554, 'eval_macro_f1': 0.661208482798357, 'eval_micro_f1': 0.7363945578231292, 'eval_pos_f1': 0.8208092485549133, 'eval_accuracy': 0.7363945578231292}
{'loss': 0.528931298828125, 'learning_rate': 0.0031972789115646263, 'epoch': 68.02721088435374}
{'epoch': 100.0}
Finished round {'eval_loss': 0.5401726365089417, 'eval_macro_f1': 0.6849089249465267, 'eval_micro_f1': 0.7491496598639455, 'eval_pos_f1': 0.8271821909783246, 'eval_accuracy': 0.7491496598639455}
{'loss': 0.52911943359375, 'learning_rate': 0.0031972789115646263, 'epoch': 6

In [87]:
len(final_results)

16

In [85]:
import pickle
pickle.dump(final_results, open('style_scaled_cv_results.pkl', 'wb'))

In [23]:
train[0]

{'feats': [0.9675265502110052,
  -1.2649550896245798,
  2.212303683527317,
  3.719023666211753,
  1.2814686886769877,
  -1.1414987273685924,
  -3.3230770917656134,
  3.2049196209819506,
  3.7505090094129336,
  2.1706627982817297,
  3.45856090373662,
  -0.2849823147028905,
  8.705847620217531,
  5.942163990529201,
  -0.4546840173262411,
  -0.15177206942306856,
  -0.508820576749366,
  0.5803780652959524,
  2.2038986093622017,
  0.5489217940868926,
  2.1022425039300585,
  1.7868556461890417,
  2.7360632898162267,
  3.5480194536898173,
  1.7509709544810568,
  2.890430818018812,
  0.7191477404999346,
  -0.1548699418786665,
  -0.0054836741303174045,
  -1.0333070154760522,
  0.1050190566699573,
  -1.4267663085313635,
  -0.9850868101582596,
  -0.4567872463878535,
  -1.1383061263440537,
  1.8382889673555245,
  -0.3334723566841886,
  -0.5478685663318728,
  -0.0011161218395672214,
  -0.6287390817290557,
  3.9937014754111946,
  3.634336596958322,
  1.133900097824702,
  0.07872022371130831,
  3.384

In [66]:
from transformers import Trainer, TrainingArguments
model = IRTNet(D_in, num_users)
args = TrainingArguments(num_train_epochs=200, output_dir="../../../../tmp", learning_rate=0.001, disable_tqdm=False, logging_steps=1000, evaluation_strategy="steps")

trainer = Trainer(model=model, train_dataset=train, args=args, eval_dataset=test, compute_metrics=do_metrics)
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [37]:
res = trainer.predict(test)

In [38]:
res

PredictionOutput(predictions=(array([[0.81114924],
       [0.70171684],
       [0.6631853 ],
       ...,
       [0.77978784],
       [0.65342414],
       [0.9524509 ]], dtype=float32), array([[ 0.29968414],
       [ 0.29968414],
       [ 0.12171861],
       ...,
       [-0.16346638],
       [-0.79377156],
       [ 1.5693771 ]], dtype=float32)), label_ids=array([1, 0, 0, ..., 0, 1, 1]), metrics={'eval_loss': 0.5182624459266663, 'eval_macro_f1': 0.6502832621256893, 'eval_micro_f1': 0.7312925170068028, 'eval_pos_f1': 0.8185993111366247, 'eval_accuracy': 0.7312925170068028})